In [1]:
import pandas as pd
import requests
import json
import os
import time

# ================= 配置区域 =================

# 1. 接口配置
API_URL = "https://www.dacoknow.com/gpt/digBeijingTag/queryCommon"
USER_KEY = "milvus_251210"
TABLE_NAME = "milvus_251210"

# 【配置】需要保存几条结果到独立的单元格
# 这里设为 3，Excel中就会生成：召回结果1, 召回结果2, 召回结果3
SAVE_TOP_K = 5 

# 2. 文件路径配置
try:
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_DIR = os.getcwd()

print(f"当前工作目录: {CURRENT_DIR}")

INPUT_FILE = os.path.join(CURRENT_DIR, "question_expanded_result_v2.xlsx")
# 输出文件改个名，避免混淆
OUTPUT_FILE = os.path.join(CURRENT_DIR, "api_retrieval_split_result_v2.xlsx")

# ===========================================

def call_search_api(query_text):
    """
    调用接口，返回一个包含前 SAVE_TOP_K 条结果的【列表】
    """
    if not query_text or pd.isna(query_text) or str(query_text).strip() == "":
        return []

    payload = {
        "userKey": USER_KEY,
        "tableName": TABLE_NAME,
        "qas": str(query_text),
        "topK": 20 # 请求多一点，取前几条
    }
    
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(API_URL, json=payload, headers=headers, timeout=30)
        
        if response.status_code != 200:
            return [f"HTTP错误: {response.status_code}"]
        
        result_json = response.json()
        
        if not result_json.get("success") and result_json.get("code") != 200:
             return [f"接口错误: {result_json.get('message', '未知')}"]

        data_list = result_json.get("result", [])
        
        retrieved_items = []
        
        if isinstance(data_list, list):
            # 取前 SAVE_TOP_K 条
            for item in data_list[:SAVE_TOP_K]:
                # 优先取 'insertContent'，其次 '切片'
                content = item.get("insertContent") or item.get("切片") or ""
                retrieved_items.append(str(content))
        else:
            retrieved_items.append(f"数据格式异常: {str(result_json)}")

        return retrieved_items

    except Exception as e:
        print(f"❌ 请求异常: {e}")
        return [f"Error: {e}"]

def main():
    print("--- 开始 API 检索 (分列展示版) ---")

    if not os.path.exists(INPUT_FILE):
        print(f"❌ 文件不存在: {INPUT_FILE}")
        return
    
    try:
        df = pd.read_excel(INPUT_FILE)
    except Exception as e:
        print(f"❌ 读取Excel失败: {e}")
        return
    
    # 初始化存储结构：为每一列准备一个空列表
    # 结构: { '方案1_召回结果1': [], '方案1_召回结果2': [], ... }
    result_columns = {}
    for i in range(1, SAVE_TOP_K + 1):
        result_columns[f'方案1_召回结果{i}'] = []
        result_columns[f'方案2_召回结果{i}'] = []

    total = len(df)
    print(f"共 {total} 条数据，开始处理...")

    for index, row in df.iterrows():
        print(f"[{index+1}/{total}] 正在请求...", end="\r")
        
        # 1. 方案1 检索
        res1_list = call_search_api(row['方案1_单句扩写'])
        
        # 2. 方案2 检索
        res2_list = call_search_api(row['方案2_潜在追问'])
        
        # 3. 填充数据到对应列
        for i in range(SAVE_TOP_K):
            col_idx = i + 1 # 1, 2, 3
            
            # 填充方案1 (如果结果不足，填空字符串)
            if i < len(res1_list):
                result_columns[f'方案1_召回结果{col_idx}'].append(res1_list[i])
            else:
                result_columns[f'方案1_召回结果{col_idx}'].append("")
            
            # 填充方案2
            if i < len(res2_list):
                result_columns[f'方案2_召回结果{col_idx}'].append(res2_list[i])
            else:
                result_columns[f'方案2_召回结果{col_idx}'].append("")

    print(f"\n请求完成，正在生成 Excel 列...")

    # 将新的列添加到 DataFrame 中
    for col_name, data_list in result_columns.items():
        df[col_name] = data_list

    # 保存
    try:
        df.to_excel(OUTPUT_FILE, index=False)
        print(f"🎉 成功！结果已保存至: {OUTPUT_FILE}")
        print(f"生成的列: {list(result_columns.keys())}")
    except PermissionError:
        print("❌ 保存失败：请关闭正在打开的 Excel 文件后重试。")

if __name__ == "__main__":
    main()

当前工作目录: d:\jupyter
--- 开始 API 检索 (分列展示版) ---
共 83 条数据，开始处理...
[83/83] 正在请求...
请求完成，正在生成 Excel 列...
🎉 成功！结果已保存至: d:\jupyter\api_retrieval_split_result_v2.xlsx
生成的列: ['方案1_召回结果1', '方案2_召回结果1', '方案1_召回结果2', '方案2_召回结果2', '方案1_召回结果3', '方案2_召回结果3', '方案1_召回结果4', '方案2_召回结果4', '方案1_召回结果5', '方案2_召回结果5']
